# Home Loan Prediction

The goal of this task is to build a predictive machine learning classifier that will use historic data of loan decisions (ground truth) to make recommended decisions for new loan applications. Along the way, you’ll get comfortable with using Jupyter Notebooks to document your work, using scikit-learn to train and test basic ML models, and start thinking about the strengths and weaknesses of models like this.

The dataset `home_loans_sample.csv` a listing of home loan applications in Washington, USA, where each row of the dataset is an individual loan application. Your goal in this assignment is to build a machine learning model that can accurately predict whether a given loan application was accepted or rejected. 

## Background
In the US (and a number of other countries), it is common for people who want to buy a home to apply for a loan (called a “mortgage”) from a bank or other lender. In order for the bank to decide whether to give them this loan, the bank will look at a number of factors to determine whether they are likely to be able to repay the loan because the bank does not want to lend money to somebody who would not be able to pay it back. These factors typically include how much money the person makes, how much money they are asking for in the loan, and a handful of other things.  Banks typically have formulas in place to determine who they will approve for a given loan and who they will deny, but there is also frequently some human judgment involved in the process.

If the person is approved for the loan, they will use the loan to purchase the home and will pay it back in small chunks every month for a set period of time (usually 15 or 30 years). If they are denied, they may have to find a less expensive home to buy or address some of the factors that led them to be denied (e.g., they might have to wait until they make more money). 

In this task, we are given a set of data about whether customers were approved or denied a loan based on a number of factors. In this assignment, your job will be to build a model that predicts whether a person will be approved or denied based on these same factors.


## Part 1: Data Exploration
The first few exercises will get you used to looking at the data using `pandas`. Pandas is a widely used library in python for manipulating data. Why? Datasets can consume a _lot_ of space in your computer's memory and traditional python data structures like lists or dictionaries will become painfully slow as we add thousands of rows of data. We use a specialized dataset library `pandas` which has a specialized data structure called a `dataframe` designed to be ultra fast & efficient. Documentation is here: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
import pandas as pd # import pandas library
df = pd.read_csv('data/home_loans_sample.csv', low_memory=False) # read the csv file into a pandas dataframe object

To understand what kind of data was collected, `pandas` has some handy commands:

- `df.head()` will show us the first 5 rows of our dataset. You can also specify the first N rows, like `df.head(18)` will show us the first 18 rows.
- `df.sample(10)` will show us 10 randomly sampled rows of our dataset (but this may not show all the columns!)
- `df.shape` will tell us how many rows and how many columns are in the dataset
- `df.columns` will list the names of all columns in the dataset
- `df.describe()` will give you summary statistics about all numerical columns in the dataset

### Question 1.a:  How many rows are in this dataset? How many columns?
_Double click this text to write your answer to the question here. Show your python work in the code block (`In [  ]:`) below:_

### Question 1.b: One of the columns in the dataset is the outcome value for each application, the value we will try to predict. Which column is that?
_Double click this text to write your answer to the question here. Show your python work in the code block below, if applicable:_

### Question 1.c: What reasons were given in this dataset for denying a loan application?
Hint: There are 3 columns in the dataset that list why a loan was denied. Try looking up the pandas command to list the unique values in a column.

_Double click this text to write your answer to the question here. Show your python work in the code block below:_

### Question 1.d: Given the denial reasons columns and the rest of the columns in this dataset, think about what information you _don't_ have about each application. Rank your top 3 _missing_ pieces of information about each application that could help you better predict the application's loan outcome.
_Double click this text to write your answer to the question here. Show your python work in the code block below, if applicable:_
#1.  
#2.  
#3. 

## Part 2: Preparing Data to Input to a Model
Here we'll start using `scikit-learn` which provides simple library calls for most things we'd like to do in a simple machine learning pipeline. If you haven't used `scikit-learn` before this tutorial may be useful to give you a sense of what the library can do: https://scikit-learn.org/stable/tutorial/basic/tutorial.html

Machine learning models can only understand data that is represented numerically, but lots of the columns in our dataset like `town_name` are text _categorical_ data. Meanwhile, many models do better when continous numerical data is within small, consistent ranges, such as all data being between -1, 0 and 1, which is definitely not the case with our thousands of dollars loan units.

So first, we will separate out our samples (called _X_) into features (sometimes called attributes) we'd like to include in our model that are categorical or continous so that we can preprocess each appropriately, separately.

In [ ]:
import sklearn # import scikit-learn
from sklearn import preprocessing # import preprocessing utilites

features_cat = ['loan_purpose_name', 'applicant_sex_name']
features_num = ['loan_amount_000s', 'applicant_income_000s']

X_cat = df[features_cat]
X_num = df[features_num]

### Part 2.A One Hot Encode categorical variables
Run the following code to One Hot Encode the categorical features:

In [ ]:
enc = preprocessing.OneHotEncoder()
enc.fit(X_cat) # fit the encoder to categories in our data 
one_hot = enc.transform(X_cat) # transform data into one hot encoded sparse array format

In [ ]:
# Finally, put the newly encoded sparse array back into a pandas dataframe so that we can use it
X_cat_proc = pd.DataFrame(one_hot.toarray(), columns=enc.get_feature_names())
X_cat_proc.head()

### Question 2.a: In your own words, how is one hot coding tranforming the categorical data? What does the term "one-hot" refer to?
_Double click this text to write your answer to the question here. Show your python work in the code block below, if applicable:_

### Part 2.B Scaling down continuous numerical data
Run the following code to normalize any continous data around the column's mean. This process will ensure that the average of that feature, such as the average amount that a person asks for in loan amount, is scaled to 0. Values less than the average will be negative numbers, and values larger than the average will be positive numbers.

In [ ]:
scaled = preprocessing.scale(X_num)
X_num_proc = pd.DataFrame(scaled, columns=features_num)
X_num_proc.head()

### Part 2.C Merge our feature sets into one sample dataset _X_ and fix NaN values
Run the code below to combine the numerical and categorical feature sets.

In [ ]:
X = pd.concat([X_num_proc, X_cat_proc], axis=1, sort=False)
X.head()

### Question 2.c The code line below removes any NaN values in our sample with 0. NaNs are missing values that a model won't be able to understand. What is the _semantic_ meaning of replacing a NaN with 0 for the categorical variables? And for the continous numerical variables? 
_Double click this text to write your answer to the question here._

In [ ]:
X = X.fillna(0) # remove NaN values

### Part 2.D Create our target array _y_ that our model will try to predict

In [ ]:
y = df['loan_approved'] # target

### Part 2.E Split our data into training, test, and validation sets
Run the code below to split the data. Both validation and test sets will be used for testing our model, but use the validation set while you are developing and improving your model, and leave the test for late stage evaluation.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_TEMP, y_train, y_TEMP = train_test_split(X, y, test_size=0.30) # split out into training 70% of our data
X_validation, X_test, y_validation, y_test = train_test_split(X_TEMP, y_TEMP, test_size=0.50) # split out into validation 15% of our data and test 15% of our data
print(X_train.shape, X_validation.shape, X_test.shape) # print data shape to check the sizing is correct

### Question 2.e:  In a  single sentence, what is the difference between train, test, and validation sets?
_Double click this text to write your answer to the question here._

## Part 3. Developing Models
Scikit-learn has a substantial library of different models we can use for classification. Below are implemented two of the most simple classification models, Logistic Regression and Dummy Classifier.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# helper method to print basic model metrics
def metrics(y_true, y_pred):
    print('Confusion matrix:\n', confusion_matrix(y_true, y_pred))
    print('\nReport:\n', classification_report(y_true, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs').fit(X_train, y_train) # first fit (train) the model
y_pred = model.predict(X_validation) # next get the model's predictions for a sample in the validation set
metrics(y_validation, y_pred) # finally evaluate performance

The Dummy Classifier is a 'dummy' because it is going to use zero machine learning, and simply predict "approve this loan" (value 1) for every loan it sees.

In [ ]:
from sklearn.dummy import DummyClassifier

approve_everyone = DummyClassifier(strategy='constant', constant = 1).fit(X_train, y_train) # first fit (train) the model
y_pred_dummy = approve_everyone.predict(X_validation) # next get the model's predictions for a sample in the validation set
metrics(y_validation, y_pred_dummy) # finally evaluate performance

### Question 3.a: Considering only the data itself, why do Logistic Regression and the Dummy Classifier perform the same? What is the semantic meaning for why Dummy Classifier has such high accuracy?
_Double click this text to write your answer to the question here._

## Part 4: Obtaining a Baseline

### Task 4.a: Create a new balanced dataset where exactly half of the samples are rejected loan applications and half are accepted loan applications.
(Hint: You may choose to do this iteratively with for..loops (which may take a _long_ time to run), or consider adapting the _Down-sample Majority Class_ code from [this link](https://elitedatascience.com/imbalanced-classes) - although do note that `balance` is not our target column and `49` is likely not the correct number of samples)

_Show your python work in the code block below:_

### Task 4.b: Below, retry training and evaluating a Logistic regression model on the updated data. What is the new performance of the model? If you were to re-run the DummyClassifier on the balanced data, what do you think would happen to its performance?
(Hint: After balancing your original dataset, you might want to repeat the data processing, feature selection, etc. process performed in Parts 2 and 3).

_Double click this text to write your answer to the question here. Show your python work in the code block below:_

### Task 4.c: Interpret the Confusion Matrix by identifying the numbers of true/false positives/negatives the code below:

In [ ]:
# Write the numbers of each in below. MAKE SURE you understand which cell in the confusion matrix has each of these.
false_negative = 0
false_positive = 0
true_negative = 0
true_positive = 0

print (false_negative, false_positive, true_negative, true_positive)

### Task 4.d: How does a *false positive* in this situation hurt _the bank_? (~20 words)

_Double click this text to write your answer to the question here._

### Task 4.e: How does a *false negative* in this situation hurt _the bank_? (~20 words)

_Double click this text to write your answer to the question here._

### Task 4.f: How does a *false positive* in this situation hurt _the customer_? (~20 words)

_Double click this text to write your answer to the question here._

### Task 4.g: How does a *false negative* in this situation hurt _the customer_? (~20 words)

_Double click this text to write your answer to the question here._

## Part 5: Your Turn!

### Task 5.a: Improving the model - why might this be difficult? 

Use your own imagination and experimentation to improve predictive performance for this task, modifying the model/algorithm choices, feature/attribute choices, and data processing however you think will be most effective. It is _very_ difficult to improve the model's performance significantly, why might that be?

Important! Don't spend too much time on this portion! Consider your ability to improve the model above the baseline after Task 4.B to be only ~10% of this assignment effort, with ~5% of that given for small improvements to performance. Thus while I encourage you to experiment, do not sink excessive time into this task. The goal is to learn about the process of machine learning model development, and some of the common pitfalls!

_Hint:_ Be sure to check out the other Supervised ML models/algorithms available from the [sci-kit learn documentation](https://scikit-learn.org/stable/user_guide.html#user-guide).

_Double click this text to write your answer to the question here. Show your python work for your best model in the python code block below:_

### Task 5.b.: What is the performance of this new model on your validation data? 

_Double click this text to write your answer to the question here. Show your python work for evaluating your best model on the validation dataset in the python code block below:_

### Task 5.c.: How does your selected model perform on the withheld test set?

_Double click this text to write your answer to the question here. Show your python work for evaluating with the test data in the python code block below:_

### Task 5.d: What other models did you try, and why might this one perform better than the others?

_Double click this text to write your answer to the question here. You can use the python cell below to show your work, as needed._

## Submit Your Assignment

Once you've completed all of the above, you're done with assignment 1! You might want to double check that your code works like you expect. You can do this by choosing "Restart & Run All" in the Kernel menu. If it outputs errors, you may want to go back and check what you've done.

Once you think everything is set, please upload your final notebook (with all of your code run and output showing), to Glow with filename [yourunixID]_haii21[assignmentnumber].ipynb, e.g., ikh1_haii21a1.ipynb.